In [3]:
import json
with open('/Users/corycates/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()



dict_keys(['client-id', 'api-key'])

In [5]:
pip install yelpapi


Note: you may need to restart the kernel to use updated packages.


In [6]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [13]:
# Load API Credentials
with open('/Users/corycates/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



In [8]:
# set our API call parameters 
LOCATION = 'Las Vegas, Nevada'
TERM = 'Sushi'


In [9]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_LAS_sushi.json"
JSON_FILE



'Data/results_in_progress_LAS_sushi.json'

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_LAS_sushi.json not found. Saving empty list to file.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [14]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [15]:
## How many results total?
total_results = results['total']
total_results



874

In [16]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [17]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



44

In [18]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [19]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/44 [00:00<?, ?it/s]

In [25]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists == True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)


In [26]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Data/results_in_progress_LAS_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_LAS_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


44

In [27]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/44 [00:00<?, ?it/s]

In [28]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,7sb2FYLS2sejZKxRYF9mtg,sakana-las-vegas-3,Sakana,https://s3-media1.fl.yelpcdn.com/bphoto/0OZzJx...,False,https://www.yelp.com/biz/sakana-las-vegas-3?ad...,4448,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 36.1171507742866, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '3949 S Maryland Pkwy', 'address2...",+17252586684,(725) 258-6684,3216.176763
1,wtcOC86mlFB_oXAKJa0aOQ,its-sushi-southwest-las-vegas,ITs SUSHI Southwest,https://s3-media2.fl.yelpcdn.com/bphoto/vLXu0k...,False,https://www.yelp.com/biz/its-sushi-southwest-l...,1123,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 36.056076, 'longitude': -115.275549}",[],$$$,"{'address1': '8410 West Warm Spring Rd', 'addr...",+17028598453,(702) 859-8453,15977.586993
2,BjrKNWhtQkedHw8hP_0Bjg,umami-las-vegas-4,Umami,https://s3-media3.fl.yelpcdn.com/bphoto/JjK4he...,False,https://www.yelp.com/biz/umami-las-vegas-4?adj...,2165,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 36.281173, 'longitude': -115.286215}","[delivery, pickup]",$$,"{'address1': '6640 N Durango Dr', 'address2': ...",+17027768627,(702) 776-8627,20149.970007
3,ghVhlFpNhfBwWDFGSlt2JA,sushi-neko-las-vegas,Sushi Neko,https://s3-media1.fl.yelpcdn.com/bphoto/ZHexhk...,False,https://www.yelp.com/biz/sushi-neko-las-vegas?...,2820,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 36.1255080044462, 'longitude': -1...",[delivery],$$,"{'address1': '5115 W Spring Mountain Rd', 'add...",+17022474241,(702) 247-4241,7109.858336
4,myFPRndhdZMKdfMZyksyxQ,its-sushi-spring-mountain-las-vegas-3,ITs SUSHI Spring Mountain,https://s3-media2.fl.yelpcdn.com/bphoto/5OQj-6...,False,https://www.yelp.com/biz/its-sushi-spring-moun...,1875,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 36.12604, 'longitude': -115.20702}",[],$$,"{'address1': '4815 Spring Mountain Rd', 'addre...",+17022471320,(702) 247-1320,6693.659586


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
869,CCVh8YKdN3c-WgqEbzya3A,market-cafe-vdara-las-vegas-3,Market Cafe Vdara,https://s3-media3.fl.yelpcdn.com/bphoto/dB8ert...,False,https://www.yelp.com/biz/market-cafe-vdara-las...,263,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",3.0,"{'latitude': 36.10939091315763, 'longitude': -...",[],$$,"{'address1': '2600 W Harmon Ave', 'address2': ...",+17025902424,(702) 590-2424,5496.707815
870,r6jbUGiyoN05mZ13apQGgw,mamas-pizza-and-bistro-las-vegas,Mama's Pizza & Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/97Y5ga...,False,https://www.yelp.com/biz/mamas-pizza-and-bistr...,334,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 36.01068985422552, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '9845 S Maryland Pkwy', 'address2...",+17028376262,(702) 837-6262,15050.419358
871,18sAHGZz-WCAFqmMNqscVw,jack-in-the-box-henderson-2,Jack in the Box,https://s3-media1.fl.yelpcdn.com/bphoto/fMHxzc...,False,https://www.yelp.com/biz/jack-in-the-box-hende...,90,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",1.5,"{'latitude': 36.0308835, 'longitude': -115.029...",[delivery],$,"{'address1': '5 S Gibson Rd', 'address2': '', ...",+17025586506,(702) 558-6506,15999.879184
872,4l1cmNrXBOLVUJR29MDp3A,subway-las-vegas-207,Subway,https://s3-media4.fl.yelpcdn.com/bphoto/gQFc06...,False,https://www.yelp.com/biz/subway-las-vegas-207?...,57,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"{'latitude': 36.2911194017872, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '7171 N Hualapai Way', 'address2'...",+17026294952,(702) 629-4952,22752.738585
873,3WipOV5JlbCEO_o0tV49yA,buffalo-wild-wings-henderson-2,Buffalo Wild Wings,https://s3-media3.fl.yelpcdn.com/bphoto/tijJZU...,False,https://www.yelp.com/biz/buffalo-wild-wings-he...,477,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",2.5,"{'latitude': 36.0032055, 'longitude': -115.111...","[delivery, pickup]",$$,"{'address1': '10271 S Eastern Ave', 'address2'...",+17024921880,(702) 492-1880,16049.847825
